#Combine target and images pandas dataframes.

##Imports.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import imageio as iio
import io

import numpy as np
import pandas as pd
import tensorflow as tf

import time
import os
import sys

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/facial_landmark_pred_example/'
sys.path.append(path)

##Read target dataframe.

In [ ]:
labels_df = pd.read_pickle(path + "helen_pickle/" + "helen_ds_target_pd.pickle")

labels_df

##Read images dataframes.

In [ ]:
dfs = []
for i in range(5):
    names_p_imgs_df = pd.read_pickle(path + "helen_pickle/" + "helen_ds_img_" + str(i+1) +".pickle")
    dfs.append(names_p_imgs_df)

dfs    

##Setup tfrecord writer.

In [ ]:
def test_dataset(dataset):
    """
    Check dataset.
    """
    for ds_img_name, ds_img, ds_coord in dataset.take(3):
        print ('{}, {}, {}'.format(type(ds_img_name), type(ds_img), type(ds_coord)))

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    print("_bytes_feature value", value)
    print("_bytes_feature tf.train.Feature", tf.train.Feature(bytes_list=tf.train.BytesList(value=[value])))
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def serialize_example(img_name, img, coords):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """  
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'img_name': _bytes_feature(img_name),
      'img'     : _bytes_feature(img),
      'coords'  : _bytes_feature(coords.numpy().tobytes()),     # Convert np.array of float32 to bytes.
  }

  # Create a Features message using tf.train.Example.
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))

  return example_proto.SerializeToString()

def tf_serialize_example(img_name, img, coords):
    tf_string = tf.py_function(
        serialize_example,
        (img_name, img, coords),      # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.

    return tf.reshape(tf_string, ()) # The result is a scalar

def write_tf_rec(dataset, path, i):   
    serialized_features_dataset = dataset.map(tf_serialize_example)
    print(serialized_features_dataset)

    # Write a TFRecord    
    #writer = tf.io.TFRecordWriter(path + "tfrec_data_" + str(i+1))
    writer = tf.data.experimental.TFRecordWriter(path + "tfrec_data/" + "tfrec_data_" + str(i+1))
    writer.write(serialized_features_dataset)    

###Merge dataframes & write tfrecords.

In [ ]:
for i, df in enumerate(dfs):
    # Merge the concatenated names_p_imgs_df with the labels_df.
    inner_join_df = pd.merge(left=labels_df, right=df, 
        left_on=['file_name_label'], right_on=['file_name'], 
        how='inner')
    
    inner_join_df = inner_join_df.drop(['file_name_label'], axis=1)

    # Select 2nd last column.
    img_names = inner_join_df.iloc[:,-2]
    # Select last column.
    imgs = inner_join_df.iloc[:,-1]    
    # Select everything except last 2 columns
    coords = inner_join_df.drop(['file_name', 'img'], axis=1)
    
    # Convert from pandas to tf.data.Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((img_names.values, imgs.values, coords.values))

    # Write to tfRecord files.
    write_tf_rec(dataset, path, i)